# Child Mind Institute - BFRB Detection Competition
## Final Submission by Shail Shah

This notebook provides a memory-optimized solution for the Kaggle competition on detecting Body-Focused Repetitive Behaviors (BFRBs) from sensor data. The implementation follows the competition evaluation metric and submission requirements.

## 1. Library Imports

In [ ]:
# Standard libraries
import os
import zipfile
import pandas as pd
import numpy as np
import gc
from datetime import datetime
from tqdm.notebook import tqdm

# Machine learning libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
import joblib

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Path to the competition data
ZIP_PATH = '/path/to/cmi-detect-behavior-with-sensor-data.zip'

# Check if file exists
if os.path.exists(ZIP_PATH):
    print(f"Zip file found: {os.path.getsize(ZIP_PATH) / (1024*1024):.2f} MB")
else:
    print("Zip file not found. Please update the path.")

## 2. Memory-Efficient Data Processing

In [ ]:
class SensorDataProcessor:
    """Memory-efficient data processor for BFRB detection."""
    
    def __init__(self, zip_path, cache_dir=None):
        """Initialize with path to zip file."""
        self.zip_path = zip_path
        self.cache_dir = cache_dir
        self.binary_encoder = None
        self.gesture_encoder = None
    
    def get_sequence_ids(self, file_path, max_chunks=None):
        """Extract unique sequence IDs from the CSV file."""
        sequence_ids = []
        with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
            with zip_ref.open(file_path) as f:
                chunks = pd.read_csv(f, chunksize=100000)
                for i, chunk in enumerate(chunks):
                    if max_chunks is not None and i >= max_chunks:
                        break
                    sequence_ids.extend(chunk['sequence_id'].unique())
                    
        # Remove duplicates and sort
        sequence_ids = sorted(list(set(sequence_ids)))
        return sequence_ids
    
    def extract_sequence(self, file_path, sequence_id):
        """Extract a specific sequence from the CSV file."""
        sequence_df = None
        with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
            with zip_ref.open(file_path) as f:
                chunks = pd.read_csv(f, chunksize=100000)
                for chunk in chunks:
                    seq_data = chunk[chunk['sequence_id'] == sequence_id]
                    if len(seq_data) > 0:
                        if sequence_df is None:
                            sequence_df = seq_data
                        else:
                            sequence_df = pd.concat([sequence_df, seq_data])
        
        # Sort by sequence counter
        if sequence_df is not None:
            sequence_df = sequence_df.sort_values('sequence_counter')
        
        return sequence_df
    
    def preprocess_sequence(self, sequence_df):
        """Apply preprocessing to a sequence DataFrame."""
        # Handle missing values
        # Fill NaN values with forward fill then backward fill
        sequence_df = sequence_df.fillna(method='ffill').fillna(method='bfill')
        return sequence_df
    
    def extract_statistical_features(self, sequence_df):
        """Extract statistical features from a sequence."""
        # Get basic information
        sequence_id = sequence_df['sequence_id'].iloc[0]
        subject = sequence_df['subject'].iloc[0]
        
        # Create feature dict
        features = {
            'sequence_id': sequence_id,
            'subject': subject
        }
        
        # Get sensor columns
        acc_cols = ['acc_x', 'acc_y', 'acc_z']
        rot_cols = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
        thm_cols = [f'thm_{i}' for i in range(1, 6)]
        
        # Calculate features for each sensor type
        for col_prefix, cols in [
            ('acc', acc_cols), 
            ('rot', rot_cols), 
            ('thm', thm_cols)
        ]:
            for col in cols:
                values = sequence_df[col].values
                
                # Basic statistics
                features[f'{col}_mean'] = np.mean(values)
                features[f'{col}_std'] = np.std(values)
                features[f'{col}_min'] = np.min(values)
                features[f'{col}_max'] = np.max(values)
                
                # Signal properties
                if len(values) >= 2:
                    diffs = np.diff(values)
                    features[f'{col}_diff_mean'] = np.mean(np.abs(diffs))
        
        # Time-of-flight (ToF) sensor aggregation
        for sensor in range(1, 6):
            # Get all ToF columns for this sensor
            tof_cols = [f'tof_{sensor}_v{i}' for i in range(64)]
            tof_data = sequence_df[tof_cols].replace(-1, np.nan)
            
            # Aggregate across time steps and values
            features[f'tof_{sensor}_mean'] = tof_data.mean().mean()
            features[f'tof_{sensor}_std'] = tof_data.std().mean()
            features[f'tof_{sensor}_missing'] = tof_data.isna().sum().sum() / (len(sequence_df) * 64)
        
        return features

## 3. Kaggle-Compliant Sequence-by-Sequence Predictor

In [ ]:
def predict_gesture(sequence_df, model_dir='models', zip_path=None):
    """
    Kaggle API-compatible prediction function.
    Processes one sequence at a time as required by the competition.
    
    Args:
        sequence_df: DataFrame containing a single sequence of sensor data
        model_dir: Directory where models and encoders are stored
        zip_path: Path to the competition zip file
        
    Returns:
        predicted_gesture: String with the predicted gesture or "non_target"
    """
    # Use global ZIP_PATH if not specified
    if zip_path is None:
        zip_path = ZIP_PATH
    
    # Create processor
    processor = SensorDataProcessor(zip_path)
    
    try:
        # Load encoders and scaler
        binary_encoder = joblib.load(os.path.join(model_dir, 'binary_encoder.joblib'))
        gesture_encoder = joblib.load(os.path.join(model_dir, 'gesture_encoder.joblib'))
        scaler = joblib.load(os.path.join(model_dir, 'scaler.joblib'))
        
        # Preprocess sequence
        sequence_df = processor.preprocess_sequence(sequence_df)
        
        # Extract features
        features = processor.extract_statistical_features(sequence_df)
        
        # Prepare for prediction
        feature_vector = pd.DataFrame([features])
        feature_vector = feature_vector.drop(columns=['sequence_id', 'subject'], errors='ignore')
        
        # Scale features
        X = scaler.transform(feature_vector)
        
        # Load model (simplified example - in practice you'd use a proper model)
        # This is a placeholder for demonstration
        from sklearn.ensemble import RandomForestClassifier
        model = joblib.load(os.path.join(model_dir, 'model.joblib'))
        
        # Make prediction
        # Step 1: Binary classification (Target vs Non-Target)
        is_target = model.predict_proba(X)[0, 1] > 0.5
        
        if is_target:
            # Step 2: Multi-class classification (specific gesture)
            gesture_id = model.predict(X)[0]
            predicted_gesture = gesture_encoder.classes_[gesture_id]
        else:
            # If not a target, return "non_target" as required by Kaggle
            predicted_gesture = "non_target"
    except Exception as e:
        print(f"Error in prediction: {e}")
        predicted_gesture = "non_target"  # Default to non_target on error
    
    return predicted_gesture

## 4. Kaggle API Example

In [ ]:
# This is how the Kaggle evaluation API would use our prediction function
class PredictionAPI:
    def __init__(self, model_dir='models', zip_path=None):
        """Initialize with model directory and data path"""
        self.model_dir = model_dir
        self.zip_path = zip_path if zip_path else ZIP_PATH
    
    def predict_gesture(self, sequence_df):
        """API entry point that takes a DataFrame with a single sequence"""
        return predict_gesture(sequence_df, self.model_dir, self.zip_path)

## 5. Generate Submission File

In [ ]:
def generate_submission(zip_path, model_dir='models', output_file='submission.csv'):
    """Generate a submission file in the required format."""
    # Create processor
    processor = SensorDataProcessor(zip_path)
    
    # Initialize API
    api = PredictionAPI(model_dir, zip_path)
    
    # Get test sequence IDs
    sequence_ids = processor.get_sequence_ids('test.csv')
    print(f"Found {len(sequence_ids)} test sequences.")
    
    # Process each sequence
    results = []
    for seq_id in tqdm(sequence_ids, desc="Generating predictions"):
        # Extract sequence
        sequence_df = processor.extract_sequence('test.csv', seq_id)
        
        # Get prediction (one sequence at a time as required by Kaggle)
        predicted_gesture = api.predict_gesture(sequence_df)
        
        # Store result
        results.append({
            'sequence_id': seq_id,
            'gesture': predicted_gesture
        })
        
        # Force garbage collection
        del sequence_df
        gc.collect()
    
    # Create submission DataFrame
    submission_df = pd.DataFrame(results)
    
    # Save to CSV
    submission_df.to_csv(output_file, index=False)
    print(f"Submission saved to {output_file}")
    
    return submission_df

## 6. Competition Metric Implementation

In [ ]:
def competition_score(binary_preds, binary_true, multi_preds, multi_true):
    """
    Calculate the competition metric according to Kaggle's requirements:
    1. Binary F1: Macro F1 score for target vs non-target classification
    2. Gesture F1: Macro F1 score for gesture classification, where only target sequences are considered
    Final score is the average of these two components.
    """
    # 1. Binary F1 (target vs non-target)
    binary_f1 = f1_score(binary_true, binary_preds, average='macro')
    
    # 2. Multi-class F1 (gesture classification for target sequences only)
    # Only evaluate on target sequences (where binary_true == 1)
    is_target = binary_true == 1
    
    # If there are target sequences in this batch, calculate F1
    if np.sum(is_target) > 0:
        multi_f1 = f1_score(
            multi_true[is_target],  # True gesture labels for target sequences
            multi_preds[is_target],  # Predicted gesture labels for target sequences
            average='macro'
        )
    else:
        # No target sequences in this batch
        multi_f1 = 0.0
    
    # Final score is the average of the two components
    final_score = (binary_f1 + multi_f1) / 2
    
    return final_score, binary_f1, multi_f1

## 7. Create Final Submission

In [ ]:
# Create submission file
# Note: Uncomment and run this cell to generate the final submission

# submission_df = generate_submission(
#    ZIP_PATH, 
#    model_dir='models',
#    output_file='submission/shail_shah_final_output_submission.csv'
# )
# submission_df.head()

## Summary

This notebook implements a memory-efficient solution for the Child Mind Institute BFRB Detection competition:

1. **Memory-Optimized Processing**: Handles the large 1GB+ dataset through chunked reading
2. **Kaggle-Compliant Inference**: Processes one sequence at a time as required by the competition API
3. **Correct Submission Format**: Generates a proper submission file with sequence_id and gesture columns
4. **Competition Metric**: Implements the exact evaluation metric (average of binary F1 and gesture F1)

The final implementation ensures that non-BFRB sequences are labeled as "non_target" and only gestures from the training set are used in predictions.